## Scenario 1: A single data scientist participating in an ML competition

MLflow setup:
* Tracking server: no
* Backend store: local filesystem
* Artifacts store: local filesystem

The experiments can be explored locally by launching the MLflow UI.

In [1]:
import mlflow

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'file:///home/ubuntu/mlops-zoomcamp2022/02-experiment-tracking/running-mlflow-examples/mlruns'


In [3]:
#mlflow.list_experiments()
mlflow.search_experiments()

[<Experiment: artifact_location='file:///home/ubuntu/mlops-zoomcamp2022/02-experiment-tracking/running-mlflow-examples/mlruns/0', creation_time=1682162880959, experiment_id='0', last_update_time=1682162880959, lifecycle_stage='active', name='Default', tags={}>]

### Creating an experiment and logging a new run

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2023/04/22 11:50:39 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.


default artifacts URI: 'file:///home/ubuntu/mlops-zoomcamp2022/02-experiment-tracking/running-mlflow-examples/mlruns/798082198168465455/6efb4701b5f64cd8afbef1b08c65ac6f/artifacts'


/home/ubuntu/.cache/pypoetry/virtualenvs/exp-tracking-Ew9w6R2h-py3.9/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [2]:
#mlflow.list_experiments()
mlflow.search_experiments()

[<Experiment: artifact_location='file:///home/ubuntu/mlops-zoomcamp2022/02-experiment-tracking/running-mlflow-examples/mlruns/798082198168465455', creation_time=1682164239929, experiment_id='798082198168465455', last_update_time=1682164239929, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='file:///home/ubuntu/mlops-zoomcamp2022/02-experiment-tracking/running-mlflow-examples/mlruns/0', creation_time=1682162880959, experiment_id='0', last_update_time=1682162880959, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [6]:
#from mlflow.tracking import MlflowClient
from mlflow import MlflowClient


client = MlflowClient()

In [7]:
from mlflow.exceptions import MlflowException

try:
    #client.list_registered_models()
    client.search_registered_models()
except MlflowException:
    print("It's not possible to access the model registry :(")

In [11]:
# Get all registered models and order them by ascending order of the names
results = client.search_registered_models(order_by=["name ASC"])
print("-" * 80)
for res in results:
    for mv in res.latest_versions:
        print("name={}; run_id={}; version={}".format(mv.name, mv.run_id, mv.version))

--------------------------------------------------------------------------------
name=scenario1; run_id=6efb4701b5f64cd8afbef1b08c65ac6f; version=1


In [13]:
def print_model_info(rm):
    print("--")
    print("name: {}".format(rm.name))
    print("tags: {}".format(rm.tags))
    print("description: {}".format(rm.description))

name = 'scenario1'
model = client.get_registered_model(name)
print_model_info(model)

--
name: scenario1
tags: {}
description: linear regression model on Iris dataset
